## Import Libraries

In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [1]:
# set files directories
from params import *

TRAIN_FILE_PATH, LANDMARK_FILES_DIR_PATH, BASE_DATASET_PATH

('C:/Users/mlamali.saidsalimo/Downloads/Hand Gesture Recognition (HGR)/asl-signs/train.csv',
 'C:/Users/mlamali.saidsalimo/Downloads/Hand Gesture Recognition (HGR)/asl-signs/train_landmark_files')

## Landmarks

![pose_landmarks](images/pose_tracking_full_body_landmarks.png)

<img src="images/hand_landmarks.png" alt="hand_landmarks" width="500">


## Load Data

In [6]:

TRAIN_TFDATASET_PATH = f"{BASE_DATASET_PATH}/train_landmark_files_preprocessed_xy/trainDataset"
VAL_TFDATASET_PATH = f"{BASE_DATASET_PATH}/train_landmark_files_preprocessed_xy/valDataset" #todo : je créer un val data set
DS_CARDINALITY = 185
VAL_SIZE  = 18 #
N_SIGNS = 250
ROWS_PER_FRAME = 543

88


In [8]:
# Set constants and pick important landmarks
LANDMARK_IDX = [0,9,11,13,14,17,117,118,119,199,346,347,348] + list(range(468,543)) #

"""
- 0 : nose
- 9 : left eye

"""

print(f"Number of landmarks : {len(LANDMARK_IDX)}")
print(f"Percentage : {len(LANDMARK_IDX)/ROWS_PER_FRAME*100:.2f}%")


Number of landmarks : 88
Percentage : 16.21%


To keep it simple, we will use the preprocessed tf.Dataset from tfdataset-of-google-isl-recognition-data.

In [16]:
def preprocess(ragged_batch : tf.RaggedTensor, labels : tf.Tensor):
    # we only keep the landmarks we are interested in (see LANDMARK_IDX)
    # tf.gather : Gather slices from ragged_batch along axis axis according to indices.
    ragged_batch = tf.gather(ragged_batch, # shape = (N_SIGNS, ROWS_PER_FRAME, 3)
                             LANDMARK_IDX, # shape = (N_LANDMARKS,)
                             axis=2)

    # remove NaNs and replace them with zeros
    ragged_batch = tf.where(tf.math.is_nan(ragged_batch), tf.zeros_like(ragged_batch), ragged_batch)
    return tf.concat([ragged_batch[...,i] for i in range(3)],-1), labels

#dataset = tf.data.Dataset.load(TRAIN_TFDATASET_PATH)
#dataset = dataset.map(preprocess)
#val_ds = dataset.take(VAL_SIZE).cache().prefetch(tf.data.AUTOTUNE)
#train_ds = dataset.skip(VAL_SIZE).cache().shuffle(20).prefetch(tf.data.AUTOTUNE)
train_ds = tf.data.Dataset.load(TRAIN_TFDATASET_PATH).map(preprocess)#.shuffle(20)
#val_ds = tf.data.Dataset.load(VAL_TFDATASET_PATH).map(preprocess)

ValueError: in user code:

    File "C:\Users\mlamali.saidsalimo\AppData\Local\Temp\ipykernel_22256\2586756040.py", line 10, in preprocess  *
        return tf.concat([ragged_batch[...,i] for i in range(3)],-1), labels

    ValueError: slice index 2 of dimension 2 out of bounds. for '{{node RaggedGetItem_2/strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=1, ellipsis_mask=2, end_mask=1, new_axis_mask=0, shrink_axis_mask=4](RaggedWhere/SelectV2, RaggedGetItem_2/strided_slice/stack, RaggedGetItem_2/strided_slice/stack_1, RaggedGetItem_2/strided_slice/stack_2)' with input shapes: [?,88,2], [3], [3], [3] and with computed input tensors: input[1] = <0 0 2>, input[2] = <0 0 3>, input[3] = <1 1 1>.


- `prefetch` is used to load the next batch while the current one is being processed. `tf.data.AUTOTUNE` is used to let tensorflow decide how many batches to prefetch.
- `cache` is used to cache the dataset in memory. This is useful when the dataset is too big to fit in memory.
- `shuffle` is used to shuffle the dataset.
